# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# sample question to use in the Gradle UI that pops up

question = """
How good at Software Development is Elijah Rwothoromo? \
He has a Wordpress site https://rwothoromo.wordpress.com/. \
He also has a LinkedIn profile https://www.linkedin.com/in/rwothoromoelaijah/. \
As well as a GitHub Profile https://www.github.com/rwothoromo/.\
What can we learn from him?
"""


In [ ]:
# imports

import re, requests, os, json, tempfile, gradio as gr, anthropic, google.generativeai, ollama
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from openai import OpenAI
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO


In [ ]:
# Load environment variables
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:8]}")
else:
    print("Anthropic API Key not set")


google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# constants

MODEL_CLAUDE = "claude-sonnet-4-20250514"
MODEL_GEMINI = "gemini-2.5-flash"
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


In [ ]:
# system messages

system_message = "You are an expert assistant. Synthesize a comprehensive answer in markdown format."
system_prompt_with_url_data = "You are an expert assistant. \
    Analyze the user's question and the provided text from relevant websites to synthesize a comprehensive answer in markdown format.\
    Provide a short summary, ignoring text that might be navigation-related."


In [ ]:
# set up environment

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


In [ ]:
# Website class for URLs to be scraped

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [ ]:
# Instantiate models with API keys from environment variables

openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
claude = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
google.generativeai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [ ]:
# To scrape data based on URLs in the user prompt

def scrape_urls(text):
    try:
        # Extract all URLs from the text string using regular expressions
        urls = re.findall(r'https?://[^\s)]+', text)
        
        if len(urls) > 0:
            scraped_content = []
            for url in urls:
                print(f"Scraping: {url}")
                try:
                    site = Website(url)
                    content = f"Content from {url}:\n---\n{site.text}\n---\n"
                    scraped_content.append(content)
                    print(f"Scraping done!")
                except Exception as e:
                    print(f"Could not scrape {url}: {e}")
                    scraped_content.append(f"Could not retrieve content from {url}.\n")
            
            return "\n".join(scraped_content)
        else:
            return None
    except Exception as e:
        print(f"Error during website scraping: {e}")
        return "Sorry, I encountered an error and could not complete scraping the website(s)."


In [ ]:
# Tool definition for scrape_urls

scraping_function = {
    "name": "scrape_urls",
    "description": "Scrapes available URLs for data to update the User prompt. Call this whenever a customer provides a URL.",
    "parameters": {
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The website URL or user prompt containing URLs."
            }
        },
        "required": ["text"]
    }
}


In [ ]:
# Instantiate the tools

# tools = [{"type": "function", "function": scraping_function}]

# Define Ollama tools
tools_gpt_ollama = [{"type": "function", "function": scraping_function}]

# Define Claude tools
tools_claude = [{
    "name": scraping_function["name"],
    "description": scraping_function["description"],
    "input_schema": scraping_function["parameters"]
}]

# Gemini tool definition must be a FunctionDeclaration object without the top-level `type` in parameters.
tools_gemini = [google.generativeai.protos.FunctionDeclaration(
    name=portable_scraping_function_definition["name"],
    description=portable_scraping_function_definition["description"],
    parameters=google.generativeai.protos.Schema(
        type=google.generativeai.protos.Type.OBJECT,
        properties={
            "text": google.generativeai.protos.Schema(
                type=google.generativeai.protos.Type.STRING,
                description=portable_scraping_function_definition["parameters"]["properties"]["text"]["description"]
            )
        },
        required=portable_scraping_function_definition["parameters"]["required"]
    )
)]


In [ ]:
# Handle multiple tools

def handle_tool_call(tool_call, user_message):
    function_name = None
    arguments = None
    tool_call_id = None
    
    # Logic for different model tool call object formats
    if isinstance(tool_call, dict) and 'function' in tool_call: # Ollama
        function_name = tool_call['function']['name']
        try:
            arguments = json.loads(tool_call['function']['arguments'])
        except (json.JSONDecodeError, TypeError):
            arguments = {'text': tool_call['function'].get('arguments', user_message)}
    elif hasattr(tool_call, 'function'): # GPT, Claude
        function_name = tool_call.function.name
        tool_call_id = getattr(tool_call, 'id', None)
        if isinstance(tool_call.function.arguments, dict):
            arguments = tool_call.function.arguments
        else:
            try:
                arguments = json.loads(tool_call.function.arguments)
            except (json.JSONDecodeError, TypeError):
                arguments = {'text': tool_call.function.arguments}
    elif hasattr(tool_call, 'name'): # Gemini
        function_name = tool_call.name
        arguments = tool_call.args

    # Fallback if arguments are not parsed correctly
    if not arguments or 'text' not in arguments:
        arguments = {'text': user_message}
    
    if function_name == "scrape_urls":
        url_scraped_data = scrape_urls(arguments['text'])
        response_content = json.dumps({"url_scraped_data": url_scraped_data})
    else:
        response_content = json.dumps({"error": f"Unknown tool: {function_name}"})

    response = {
        "role": "tool",
        "content": response_content,
        "tool_call_id": tool_call_id
    }
    return response


In [ ]:
# Audio output

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)


In [ ]:
# To transcribe an audio prompt/input to text

def transcribe_audio(audio_file):
    if audio_file is None:
        return ""
    
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmpfile:
        audio = AudioSegment.from_file(audio_file, format="wav")
        audio.export(tmpfile.name, format="wav")
        
        with open(tmpfile.name, "rb") as audio_file_obj:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file_obj
            )
        return transcript.text


In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:", scale=4)
        submit_btn = gr.Button("Submit", scale=1)
    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak to our AI Assistant", scale=4)
        submit_audio_btn = gr.Button("Submit Audio", scale=1)

    with gr.Row():
        models = ["Claude", "Gemini", "GPT", "Ollama"]
        model_dropdown = gr.Dropdown(
            label="Select a model",
            choices=models,
            value=models[2]
        )

        audio_options = ["Yes", "No"]
        audio_dropdown = gr.Dropdown(
            label="Select whether to respond with audio",
            choices=audio_options,
            value=audio_options[1]
        )
        
    with gr.Row():
        clear = gr.Button("Clear")

    def user_message_updater(user_message, history):
        return "", history + [[user_message, None]]

    def chat_with_assistant(history, target_model, use_audio_output):
        messages = []
        for msg_user, msg_assistant in history:
            messages.append({"role": "user", "content": msg_user})
            if msg_assistant:
                messages.append({"role": "assistant", "content": msg_assistant})
        
        user_message = history[-1][0]
        final_response_content = ""
        
        if target_model == "Claude":
            response = claude.messages.create(
                model=MODEL_CLAUDE,
                max_tokens=200,
                temperature=0.7,
                system=system_prompt_with_url_data,
                messages=messages,
                tools=tools_claude,
            )
            
            tool_calls = [content_block for content_block in response.content if content_block.type == "tool_use"]
            if tool_calls:
                tool_use = tool_calls[0]
                tool_output_content = scrape_urls(tool_use.input["text"])
                
                messages.append({"role": "assistant", "content": response.content})
                messages.append({
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_output_content
                        }
                    ]
                })

                response = claude.messages.create(
                    model=MODEL_CLAUDE,
                    max_tokens=200,
                    temperature=0.7,
                    system=system_prompt_with_url_data,
                    messages=messages,
                )
            final_response_content = response.content[0].text

        elif target_model == "Gemini":
            messages_gemini = []
            for m in history:
                messages_gemini.append({"role": "user", "parts": [{"text": m[0]}]})
                if m[1]:
                    messages_gemini.append({"role": "model", "parts": [{"text": m[1]}]})
            
            model = google.generativeai.GenerativeModel(
                model_name=MODEL_GEMINI,
                system_instruction=system_message,
                tools=tools_gemini
            )
            
            chat = model.start_chat(history=messages_gemini[:-1])
            response = chat.send_message(messages_gemini[-1])

            # Check if the response is a tool call before trying to extract text
            if response.candidates[0].content.parts[0].function_call:
                tool_call = response.candidates[0].content.parts[0].function_call
                response_tool = handle_tool_call(tool_call, user_message)

                tool_response_content = json.loads(response_tool["content"])
                tool_response_gemini = {
                    "role": "tool",
                    "parts": [{
                        "function_response": {
                            "name": tool_call.name,
                            "response": tool_response_content
                        }
                    }]
                }
                
                # Send the tool output back and get a new response
                response = chat.send_message(tool_response_gemini)
                final_response_content = response.text
            else:
                # If the original response was not a tool call, get the text directly
                final_response_content = response.text

        elif target_model == "Ollama":
            messages_ollama = [{"role": "system", "content": system_message}] + messages
            response = ollama.chat(
                model=MODEL_LLAMA,
                messages=messages_ollama,
                stream=False,
                tools=tools_gpt_ollama,
            )

            if 'tool_calls' in response['message'] and response['message']['tool_calls']:
                response_tool = handle_tool_call(response['message']['tool_calls'][0], user_message)
                messages_ollama.append({"role": "assistant", "content": response['message']['content'], "tool_calls": response['message']['tool_calls']})
                messages_ollama.append(response_tool)
                
                response = ollama.chat(
                    model=MODEL_LLAMA,
                    messages=messages_ollama,
                    stream=False,
                )
            final_response_content = response['message']['content']
        
        else: # Assuming GPT is default
            messages_gpt = [{"role": "system", "content": system_message}] + messages
            response_stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages_gpt, stream=True, tools=tools_gpt_ollama)
            final_response_content = ""
            for chunk in response_stream:
                content = chunk.choices[0].delta.content or ""
                tool_calls_chunk = chunk.choices[0].delta.tool_calls
                if content:
                    final_response_content += content
                
                if tool_calls_chunk:
                    tool_call = tool_calls_chunk[0]
                    response_tool = handle_tool_call(tool_call, user_message)
                    
                    messages_gpt.append({"role": "assistant", "tool_calls": [tool_call]})
                    messages_gpt.append(response_tool)
                    
                    response_stream_after_tool = openai.chat.completions.create(model=MODEL_GPT, messages=messages_gpt, stream=True)
                    for chunk_after_tool in response_stream_after_tool:
                        final_response_content += chunk_after_tool.choices[0].delta.content or ""
                    break

        history[-1][1] = final_response_content
        
        if use_audio_output != "No":
            talker(final_response_content)

        return history

    def transcribe_and_chat(audio_file, history, target_model, use_audio_output):
        if audio_file:
            transcribed_text = transcribe_audio(audio_file)
            new_history = history + [[transcribed_text, None]]
            return chat_with_assistant(new_history, target_model, use_audio_output)
        else:
            return history

    entry.submit(
        user_message_updater,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot],
        queue=False
    ).then(
        chat_with_assistant,
        inputs=[chatbot, model_dropdown, audio_dropdown],
        outputs=[chatbot]
    )

    submit_btn.click(
        user_message_updater,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot],
        queue=False
    ).then(
        chat_with_assistant,
        inputs=[chatbot, model_dropdown, audio_dropdown],
        outputs=[chatbot]
    )

    audio_input.stop(
        transcribe_and_chat,
        inputs=[audio_input, chatbot, model_dropdown, audio_dropdown],
        outputs=[chatbot],
        queue=False
    )

    submit_audio_btn.click(
        transcribe_and_chat,
        inputs=[audio_input, chatbot, model_dropdown, audio_dropdown],
        outputs=[chatbot],
        queue=False
    )
    
    clear.click(lambda: None, inputs=None, outputs=[chatbot], queue=False)

ui.launch(inbrowser=True)
